## A3 Machine Learning
Done By:
- Nikhil Tyagi (B00809791)
- Aniruddha Chitley (B00808320)
- Nitish Bhardwaj (B00811535)

In [4]:
# Imports
import matplotlib.pyplot as plt
import random, pdb
import numpy as np
import tensorflow as tf
import time
import pandas as pd

#svm imports
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

#Neural net import
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

#Linear classifier
from sklearn.linear_model import LogisticRegression

# Random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
# Fetching MNIST Data (Code from Assignment 1)
mnist = tf.keras.datasets.mnist
(train_data, train_labels),(test_data, test_labels) = mnist.load_data()

print("Number of training examples\t",len(train_data))
print("Number of test instances\t",len(test_data))
print("Shape of train_data\t\t",train_data.shape)
print("Shape of train_labels\t\t",train_labels.shape)
print("Shape of test_data\t\t",test_data.shape)
print("Shape of test_labels\t\t",test_labels.shape)

Number of training examples	 60000
Number of test instances	 10000
Shape of train_data		 (60000, 28, 28)
Shape of train_labels		 (60000,)
Shape of test_data		 (10000, 28, 28)
Shape of test_labels		 (10000,)


In [6]:
# Vectorize test and training data
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1] * test_data.shape[2]).astype('float32')
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1] * train_data.shape[2]).astype('float32')

print('New shape of test_data', test_data.shape)
print('New shape of train_data', train_data.shape)

New shape of test_data (10000, 784)
New shape of train_data (60000, 784)


In [7]:
# Scale data to 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.fit_transform(test_data)

### SVC - Support Vector Classifier

In [21]:
# Grid search for SVC on MNIST
parameters = {
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'C':[0.001, 0.1, 100, 10e5],
    'gamma':[10,1,0.1,0.01]
}
grid_svc = GridSearchCV(svm.SVC(), param_grid=parameters, cv=3, n_jobs=-1, return_train_score=True)

In [23]:
grid_svc.fit(train_data[:10000], train_labels[:10000])

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'kernel': ['poly', 'rbf', 'sigmoid'], 'C': [0.001, 0.1, 100, 1000000.0], 'gamma': [10, 1, 0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [24]:
print(grid_svc.score(test_data, test_labels))
print(grid_svc.best_params_)

df = pd.DataFrame(grid_svc.cv_results_)
df.to_pickle('mnist_svc.pkl')

0.9659
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [26]:
df = pd.read_pickle('mnist_svc.pkl')
df[['param_kernel','param_C', 'param_gamma', 'mean_test_score']]

,param_kernel,param_C,param_gamma,mean_test_score
0,poly,0.001,10,0.9472
1,rbf,0.001,10,0.1127
2,sigmoid,0.001,10,0.1127
3,poly,0.001,1,0.9472
4,rbf,0.001,1,0.1127
5,sigmoid,0.001,1,0.1127
6,poly,0.001,0.1,0.9321
7,rbf,0.001,0.1,0.1127
8,sigmoid,0.001,0.1,0.1127
9,poly,0.001,0.01,0.1131


### Random Forest Classifier

In [17]:
# Random Forests, grid search to find best parameter and run cross validation
parameters = {
    'n_estimators': [100],
    'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)]
}
grid_rfc = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=2, n_jobs=-1, return_train_score=True)

In [18]:
grid_rfc.fit(train_data, train_labels)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [100], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [19]:
print(grid_rfc.score(test_data, test_labels))
print(grid_rfc.best_params_)

df = pd.DataFrame(grid_rfc.cv_results_)
df.to_pickle('mnist_rfc.pkl')

0.9696
{'max_depth': 80, 'n_estimators': 100}


In [20]:
df = pd.read_pickle('mnist_rfc.pkl')
df[['param_n_estimators','param_max_depth', 'mean_fit_time', 'mean_test_score']]

,param_n_estimators,param_max_depth,mean_fit_time,mean_test_score
0,100,10,25.584607,0.942350
1,100,20,34.939611,0.961650
2,100,30,36.224180,0.961817
3,100,40,36.129935,0.961550
4,100,50,39.031673,0.961400
5,100,60,39.911803,0.961850
6,100,70,39.959677,0.962167
7,100,80,39.967157,0.962500
8,100,90,35.783377,0.961567
9,100,100,31.846899,0.961567


### Linear Classifier

In [8]:
# Linear Classifier
parameters = {
    'solver': ['sag', 'lbfgs'],
    'multi_class': ['ovr']
}
grid_lr = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=2, n_jobs=-1, return_train_score=True)

In [9]:
grid_lr.fit(train_data, train_labels)

c:\users\nikhi\appdata\local\conda\conda\envs\ml-class\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'solver': ['sag', 'lbfgs'], 'multi_class': ['ovr']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [12]:
print(grid_lr.score(test_data, test_labels))
print(grid_lr.best_params_)

df = pd.DataFrame(grid_lr.cv_results_)
df.to_pickle('mnist_lr.pkl')

0.9201
{'multi_class': 'ovr', 'solver': 'sag'}


In [14]:
df = pd.read_pickle('mnist_lr.pkl')
df[['param_solver','param_multi_class', 'mean_fit_time', 'mean_test_score']]

,param_solver,param_multi_class,mean_fit_time,mean_test_score
0,sag,ovr,382.959543,0.911417
1,lbfgs,ovr,97.299586,0.911383


### Neural Network

In [6]:
# Neural Nets
parameters = {
    'activation': ['logistic'],
    'solver': ['sgd'],
    'max_iter': [100],
    'learning_rate_init': [0.001, 0.01],
    'hidden_layer_sizes': [(10, 15), (100, 150), (550, 200)]
}
grid_nn = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1, cv=5, return_train_score=True)

In [ ]:
grid_nn.fit(train_data, train_labels)

In [ ]:
print(grid_nn.score(test_data, test_labels))
print(grid_nn.best_params_)

df = pd.DataFrame(grid_nn.cv_results_)
df.to_pickle('mnist_nn.pkl')

In [16]:
df = pd.read_pickle('mnist_nn.pkl')
df[['param_learning_rate_init','param_hidden_layer_sizes', 'mean_fit_time', 'mean_test_score']]

,param_learning_rate_init,param_hidden_layer_sizes,mean_fit_time,mean_test_score
0,0.001,"(10, 15)",120.627461,0.663233
1,0.01,"(10, 15)",122.322817,0.921450
2,0.001,"(100, 150)",442.548446,0.836833
3,0.01,"(100, 150)",446.719322,0.954433
4,0.001,"(550, 200)",1911.675161,0.866133
5,0.01,"(550, 200)",1900.752605,0.938700


<b>Q:</b> Which algorithm (and the corresponding settings) performs best?<br>
<b>A:</b>
For the various settings tried above, best performance is given by Neural Networks with 97.15%.
<br><br>

<b>Q:</b> Which algorithm was easiest to tune?<br>
<b>A:</b>
Random forest classifier took least time to tune as compared to other algorithms and presented with a accuracy of 96.96%.
<br><br>


<b>Q:</b> Be sure to show the various hyperparameter settings you tried. <br>
<b>A:</b>
Hyperparameter settings are shown in parameters variable under each algorithm used.
<br><br>


<b>Q:</b> For each algorithm, describe what you learnt from the various experiments.<br>
<b>A:</b> Our findings for each algorithm:<br>
<b>Support Vector Classifier</b>:
- Best accuracy of SVC is: 96.59%
- Best config is {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
- It was taking a lot of time for svc classifier to work on whole dataset hence we tried training for 10,000 data elements only. We left for it to work on server (bluenose) for 3 days with no result.
- Reason for high regularization parameter(C) can be because of 784 dimensions in mnist data.
- 

<b>Random Forest Classifier</b>:
- Best accuracy is: 96.96%
- Best config is {'max_depth': 80, 'n_estimators': 100}.
- Random forest classifier classifies mnist based on various decision trees. As 80 depth tree is producing the best results that can signify that out of 784 dimensions, decisions on 80 different dimensions can successfully classify digits.

<b>Linear Classifier</b>:
- Best accuracy is: 92.01 %
- Best config is {'multi_class': 'ovr', 'solver': 'sag'}
- Linear classifier for mnist data has lowest accuracy among its peers. Although it is above 90%.
- This signifies that in 784 dimensions their exists hyperplanes that can successfully classify various digits in the data.

<b>Neural Network</b>:
- Neural Networks best accuracy is of 97.15%. 
- Best config is {'activation': 'logistic', 'hidden_layer_sizes': (100,150), 'learning_rate_init':0.01, 'max_iter': 100, 'solver': 'sgd'}.
- It takes a long time to classify mnist data using neural networks.
- For neural networks, two hidden layers works best for mnist data.
<br><br>
 